In [2]:
import numpy as np
from utils import parse_csi, get_data_files, generate_labels, split_csi
from nn_utils import CSIModel, train, evaluate, ComplexCSIModel


import numpy as np
import torch
from torch.utils.data import DataLoader, random_split, TensorDataset
import torch.nn as nn
import torch.optim as optim

import os 

In [3]:

file_paths = get_data_files(data_dir = '../data/')
file_paths = sorted(file_paths)
print(file_paths)

['../data/room_doorclosed.csv', '../data/room_doorclosed2.csv', '../data/room_dooropen.csv', '../data/room_playcomputer.csv']


In [5]:
macs = ['A0:A3:B3:AA:76:38']

data, found_macs = [], [] 
for file_path in file_paths: 
    csi, mac = parse_csi(file_path, macs)
    print(csi.shape)
    csi = split_csi(csi, 3000) 
    if file_path.split('/')[-1].split('.')[0][-1].isdigit():
        data[-1] = np.concatenate((data[-1], csi), axis = 0)
        found_macs[-1] = np.concatenate((found_macs[-1], mac), axis = 0) 
    else:
        data.append(csi)
        found_macs.append(mac)

reading file:  ../data/room_doorclosed.csv
(3866803, 2)
reading file:  ../data/room_doorclosed2.csv
(3134773, 2)
reading file:  ../data/room_dooropen.csv
(8210781, 2)
reading file:  ../data/room_playcomputer.csv
(7561713, 2)


In [6]:
labels = generate_labels(data) 

data = np.concatenate(data, axis = 0)
labels = np.concatenate(labels, axis = 0)
data.shape, labels.shape

((7588, 3000, 2), (7588,))

In [7]:

t_data, t_labels = torch.tensor(data, dtype = torch.float32), torch.tensor(labels, dtype = torch.long)
dataset = TensorDataset(t_data, t_labels)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle = True)

input_size, hidden_size, output_size = 2, 10, len(np.unique(labels)) 
model = ComplexCSIModel(input_size, hidden_size, num_layers = 10, output_size = output_size, dropout_rate = 0.2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)

train(model, train_loader, criterion, optimizer, num_epochs = 10) 


Epoch 1/10, Loss: 1.0978391816741542
Epoch 2/10, Loss: 1.0971755711655868
Epoch 3/10, Loss: 1.0967040463497764
Epoch 4/10, Loss: 1.0968593402912743
Epoch 5/10, Loss: 1.0969389413532458
Epoch 6/10, Loss: 1.0968270213980424
Epoch 7/10, Loss: 1.0969944382968702
Epoch 8/10, Loss: 1.0968635747307225
Epoch 9/10, Loss: 1.096896473357552
Epoch 10/10, Loss: 1.096853524760196


In [8]:
tars, preds = evaluate(model, val_loader)

Accuracy on validation set: 36.23%


In [9]:
# calculate metrics 
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(tars, preds)
print(cm)
print(classification_report(tars, preds))


[[  0 460   0]
 [  0 550   0]
 [  0 508   0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       460
           1       0.36      1.00      0.53       550
           2       0.00      0.00      0.00       508

    accuracy                           0.36      1518
   macro avg       0.12      0.33      0.18      1518
weighted avg       0.13      0.36      0.19      1518



/opt/anaconda3/envs/rem/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/rem/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/rem/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
